In [ ]:
knitr::opts_chunk$set(
  collapse = TRUE,
  comment = "#>"
)

This vignette demonstrates how to use the `ffaframework` package to perform flood frequency analysis on a *stationary* dataset.

The results are expressed in terms of *return periods* and *return levels*:

- The *return period* of a flood is the expected number of years between streamflow events of equal or greater severity.
- The *return level* is the magnitude of the streamflow event, measured in $\text{m}^3/\text{s}$.

## Setup

In [ ]:
library(ffaframework)

csv_path <- system.file("extdata", "Application_1.csv", package = "ffaframework")
df <- read.csv(csv_path)
df <- subset(df, !is.na(max)) # Remove missing values

head(df)

## Distribution Selection

First, a suitable probability distribution for the data is selected using the method of L-moments.

1. `select_ldistance` chooses the distribution whose theoretical L-skewness ($\tau_3$) and L-kurtosis ($\tau_4$) has the smallest Euclidean distance from the sample point.
2. `select_lkurtosis` matches the theoretical L-kurtosis to the sample L-kurtosis.
3. `select_zstatistic` uses a fitted 4-parameter Kappa distribution to estimate the sampling distribution of the L-kurtosis and selects the distribution with the smallest z-statistic.

In [ ]:
selection <- select_ldistance(df$max)

print(selection$recommendation)

plot_lmom_diagram(selection)

**Conclusion**: Use the generalized extreme value (GEV) distribution.

**Note**: For information about the other distributions, see the `selection$metrics` item.

You can find more information about the probability distributions supported by the framework [here](https://rileywheadon.github.io/ffa-docs/distributions/).

## Parameter Estimation

The `ffaframework` package provides two methods for parameter estimation.
See [here](https://rileywheadon.github.io/ffa-docs/parameter-estimation/) for more information.

- `fit_lmom_*`: L-moments parameter estimation for each distribution.
- `fit_maximum_likelihood`: Maximum likelihood and generalized maximum likelihood.

In [ ]:
params <- fit_lmom_gev(df$max)

print(params)

**Conclusion**: The $\text{GEV}(1600.22, 616.67, 0.12)$ distribution will be used.

## Uncertainty Quantification

Given a fitted probability distribution it is easy to estimate the return levels. 
However, point estimates alone are insufficient; confidence intervals should also be reported.
The `uncertainty_bootstrap` function performs uncertainty quantification using the sample bootstrap method. It requires three arguments:

- `data`: A vector of annual maximum streamflow observations.
- `model`: A three-letter code for a probability distribution (ex. `"GEV"`).
- `method`: A parameter estimation method. Must be `"L-moments"`, `"MLE"`, or `"GMLE"`.

By default, return levels are computed 2-, 5-, 10-, 20-, 50-, and 100- year return periods. 

In [ ]:
uncertainty <- uncertainty_bootstrap(df$max, "GEV", "L-moments")

print(uncertainty[[1]]$estimates)

plot_sffa(uncertainty)

**Example Conclusion**: Every 10 years, we can expect a streamflow event of $3194.788\text{m}^3/\text{s}$ or greater.

## Model Assessment

Model performance is assessed using `model_diagnostics`, which reports a collection of assessment statistics about the flood frequency analysis.
`plot_model_diagnostics` compares the empirical plotting positions (the "Observed Quantiles") and the predictions of the parametric model (the "Model Quantiles"). 
The black line represents a perfect 1:1 correspondence between the model and the data.

In [ ]:
diagnostics <- model_diagnostics(df$max, "GEV", params, uncertainty)

plot_model_diagnostics(diagnostics)

**Conclusion**: The parametric model generally matches the plotting positions. There is a small positive bias around $4000\text{m}^3/\text{s}$.